# 함수형 API로 복잡한 신경망 만들기

In [1]:
import tensorflow as tf
import numpy as np

/Users/barrett11357/coding/handson_ml_--/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [9]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

datasets=fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(datasets.data, datasets.target, random_state=42)#훈련세트와 테스트세트로 분할
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)#훈련세트에서 검증세트 분리

## 와이드 & 딥 신경망 구현

입력의 일부 또는 전체가 출력 층에 바로 연결되는 와이드 & 딥 신경망을 구현해볼거다.
이 구조를 사용하면 깊게 쌓인 층을 이용해 복잡한 패턴과 짧은 경로를 사용한 간단한 규칙을 모두 학습할 수 있다.

함수형 API를 사용하면 이런 순차적이지 않은 신경망을 구현할 수 있다.

### 모든 특성을 짧은 경로로 연결

In [10]:
#먼저 필요한 레이어들 준비한다.
norm_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(50, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(50, activation="relu")
concat_layer = tf.keras.layers.Concatenate()#얘가 여러 입력 텐서를 하나의 입력 텐서로 연결시키는 역할을 한다. 
output_layer = tf.keras.layers.Dense(1)

#이 밑 부분부터가 특이하다. 각각의 레이어들을 함수처럼 사용한다.
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
nomalized = norm_layer(input_)
hidden1 = hidden_layer1(nomalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([nomalized, hidden2])#concat layer를 사용해서 입력과 두번째 은닉층의 출력을 연결시킨다.
output = output_layer(concat)

model = tf.keras.Model(inputs=[input_], outputs=[output])

### 일부 특성만 짧은 경로로 연결

이 경우에는 입력을 여러개 사용하면 된다.

그리고 이번에는 Dense층의 생성과 호출을 동시에 해보면서 코드를 좀 더 간략하게 짜 볼 것이다.
Nomalization층은 fit()을 호출하기 전에 adapt()를 호출해서 데이터에 적응시켜야 하기 때문에 참조값을 따로 유지해야 해서 이런 식으로 쓸 수는 없다.

In [11]:
#짧은 경로
input_wide = tf.keras.layers.Input(shape=[5])#특성인덱스 0부터 4까지, 오해하면 안되는게 단순히 입력의 크기가 5라는 얘기다. 슬라이싱 해서 입력하는 것은 모델 밖에서 하게 되는 일이다.
#깊은 경로
input_deep = tf.keras.layers.Input(shape=[6])#특성인덱스 2부터 7까지

#Nomalization층은 fit()을 호출하기 전에 adapt()를 호출해서 데이터에 적응시켜야 하기 때문에 참조값을 따로 유지해야 한다.
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()


norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(50, activation="relu")(hidden1)

concat = tf.keras.layers.Concatenate()([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)


model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

좀 더 부연설명을 하자면 norm_wide, hidden1, concat, output 같은 변수들은 지금 레이어를 담고 있는게 아니다.
레이어 객체 자체가 아니라, 해당 레이어를 입력 텐서에 적용한 후 생성된 출력 텐서를 담고 있는 변수이다. 

예를 들어 `hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)` 이 코드에서
`tf.keras.layers.Dense(50, activation="relu")` 이 코드로 Dense 객체를 생성하고, 
생성된 Dense 레이어 객체를 norm_deep 텐서에 적용하여 출력 텐서를 계산하고, 그 결과를 hidden1에 저장한 것이다. 

컴파일은 앞에서 했던 것과 똑같이 하면 되고,

모델을 훈련할 때 하나의 입력 행렬 X_train을 넣으면 되는게 아니고
입력 마다 하나씩 행렬의 튜플 (X_train_wide, X_train_deep)을 넣어줘야 한다. 
검증과 테스트 데이터에도 마찬가지이다. 

In [13]:
model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=[tf.keras.metrics.RootMeanSquaredError()])


In [14]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]
X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]

norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=1, validation_data=((X_valid_wide, X_valid_deep), y_valid))

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 646us/step - loss: 2.2329 - root_mean_squared_error: 1.4581 - val_loss: 1.7375 - val_root_mean_squared_error: 1.3181


### 여러개의 출력 사용하기

보조출력을 만드는 것은 쉬워용. 그냥 출력층을 하나 만들고, Model객체를 만들 때 outputs리스트에 그 출력층을 추가해주기만 하면 된다.

In [28]:
#위의 일부특성만 짧은 경로로 연결하는 코드를 그대로 가져옴.
input_wide = tf.keras.layers.Input(shape=[5])#특성인덱스 0부터 4까지, 오해하면 안되는게 단순히 입력의 크기가 5라는 얘기다. 슬라이싱 해서 입력하는 것은 모델 밖에서 하게 되는 일이다.
input_deep = tf.keras.layers.Input(shape=[6])#특성인덱스 2부터 7까지

#Nomalization층은 fit()을 호출하기 전에 adapt()를 호출해서 데이터에 적응시켜야 하기 때문에 참조값을 따로 유지해야 한다.
norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()
norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)
hidden1 = tf.keras.layers.Dense(50, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(50, activation="relu")(hidden1)
concat = tf.keras.layers.Concatenate()([norm_wide, hidden2])
output = tf.keras.layers.Dense(1, name="output")(concat)#출력 층에 따로 이름 붙여줌.
aux_output = tf.keras.layers.Dense(1, name="aux_output")(hidden2)#새로 추가한 보조출력

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output, aux_output])

각 출력은 자신만의 손실함수를 가져야 한다.
컴파일 할 때 손실의 리스트를 전달해주면 된다.
기본적으로 케라스는 나열된 손실을 모두 더하여 **최종**손실을 구해 훈련에 사용한다고 한다.

In [18]:
model.compile(loss=["mse","mse"], 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              metrics=["RootMeanSquaredError", "RootMeanSquaredError"])#출력이 2개 이므로 메트릭스도 2개를 넣어줘야 한다.

이떄 손실함수를 위와같이 리스트 말고, 출력이름-손실함수 쌍의 딕셔너리로 전달할 수도 있다.

In [35]:
model.compile(loss={"output":"mse", "aux_output":"mse"}, 
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              metrics=["RootMeanSquaredError", "RootMeanSquaredError"])


In [38]:
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_22      │ (None, 6)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_22    │ (None, 6)         │         13 │ input_layer_22[0… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_21      │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_43 (Dense)    │ (None, 50)        │        350 │ normalization_22… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_21    │ (None, 5)         │         11 │ input_layer_21[0… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_44 (Dense)    │ (None, 50)        │      2,550 │ dense_43[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_13      │ (None, 55)        │          0 │ normalization_21… │
│ (Concatenate)       │                   │            │ dense_44[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         56 │ concatenate_13[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ aux_output (Dense)  │ (None, 1)         │         51 │ dense_44[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 9,047 (35.35 KB)

 Trainable params: 3,007 (11.75 KB)

 Non-trainable params: 24 (104.00 B)

 Optimizer params: 6,016 (23.50 KB)

예측을 수행할 때는 출력이 여러개이기 때문에 각 출력에 대해서 따로따로 레이블을 제공해야 한다. 
다만 지금 코드에서는 주출력과 보조출력이 같은 것을 예측하므로 동일한 레이블을 사용한다. 

여기에서도 그냥 튜플로 전달하거나, 출력이름과 매칭한 딕셔너리로 전달할 수도 있다.

In [37]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

model.fit((X_train_wide, X_train_deep),(y_train, y_train),
          epochs=1,
          validation_data=((X_valid_wide, X_valid_deep),(y_valid, y_valid)))

363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 895us/step - aux_output_RootMeanSquaredError: 0.7511 - aux_output_loss: 0.5642 - loss: 1.0218 - output_RootMeanSquaredError: 0.6763 - output_loss: 0.4576 - val_aux_output_RootMeanSquaredError: 1.4802 - val_aux_output_loss: 2.1899 - val_loss: 4.1369 - val_output_RootMeanSquaredError: 1.3950 - val_output_loss: 1.9452
